<a href="https://colab.research.google.com/github/biwanun1690/Resume-Builder-and-Job-Selection-Algorithm-1511/blob/Algorithm-resume-vacancy/transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
import random
import re
import nltk
!pip install translate
!pip install googletrans
!pip install -U textblob
!python -m textblob.download_corpora
!pip install gensim
!pip install nltk pymorphy2
!pip install pymystem3
import googletrans
import asyncio
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from googletrans import Translator
from pymystem3 import Mystem
from nltk.corpus import stopwords

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 79.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d2c0ed34ab6b8b209e8d9f43287

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data = ['Чижов Андрей Тимофеевич', '2003-02-21', 'мужской', 'num-ugideju67@yandex.ru', '(8549)595-64-54', 'г. Москва ул.Тартар д.24 к.1',
         'Высшее', 'Высшая академическая школа графического дизайна', '3D графика', '2021',
         'Средний работник с опытом', 'Дизайн', '3D дизайнер',
         'от 1 до 3 лет', 'Написал статью, опубликованную в профессиональном журнале', 'ProtoPuf', 'Создание визульных спецэффектов, использование современных технологий работы создания моделей.']
index = ['fullname', 'date_of_birth', 'gender', 'email', 'phone', 'address',
         'education', 'institution', 'faculty', 'year_graduation',
         'status', 'area_work', 'branch',
         'experience', 'achievements', 'company',
         'skills']
resume = pd.Series(data, index=index)

data = ['Дизайнер-художник', 'Дизайн', '100000', '150000', 'г. Москва ул.Тушканчик д.10', 'Надежда Луна',
         'Понимания специфики различных материалов и современных технологий нанесения (офсетная и цифровая печать, предпечатная подготовка).',
         'Отвечать за разработку визуальных концепций дизайна и их реализацию для различных носителей (полиграфия, текстиль, фарфор и тд).',
         '76873642384', 'Полный день', np.NaN, 'Сменный график', '5 часов', False, 'Графический дизайнер', 'от 1 до 3 лет',
        'Проектная работа', False]

index = ['name',	'area_work',	'salary_from', 	'salary_to',	'addres',	'employer',	'requirement',	'responsibility',
         'contacts',	'schedule',	'working_days','	work_format',	'working_hours',	'night_shifts',
         'professional_roles',	'experience', 'employment', 'internship']
vacancy = pd.Series(data, index=index)

print(resume)
print()
print(vacancy)

resume_text = resume['achievements'] + '. ' + resume['skills']
vacancy_text = vacancy['requirement'] + ' ' +  vacancy['responsibility']

fullname                                     Чижов Андрей Тимофеевич
date_of_birth                                             2003-02-21
gender                                                       мужской
email                                        num-ugideju67@yandex.ru
phone                                                (8549)595-64-54
address                                 г. Москва ул.Тартар д.24 к.1
education                                                     Высшее
institution          Высшая академическая школа графического дизайна
faculty                                                   3D графика
year_graduation                                                 2021
status                                     Средний работник с опытом
area_work                                                     Дизайн
branch                                                   3D дизайнер
experience                                             от 1 до 3 лет
achievements       Написал статью,

In [ ]:
from translate import Translator

translator = Translator(from_lang="ru", to_lang="en")
skills = translator.translate(skills)
need_skills = translator.translate(need_skills)
print(skills)
print(need_skills)

Creation of visual special effects, the use of modern technologies for creating models.
Understanding the specifics of various materials and modern application technologies (offset and digital printing, prepress). Responsible for the development of visual design concepts and their implementation for various media (printing, textiles, porcelain, etc.).


In [ ]:
def processing(text):
  # Регулярные выражения
  pattern = r"[^\w]"
  text = re.sub(pattern, " ", text)

  # Лемматезация
  m = Mystem()
  lemmas = m.lemmatize(text)
  text = "".join(lemmas).strip()

  # Токенизация
  text = nltk.sent_tokenize(text)
  text = [nltk.word_tokenize(sentence) for sentence in text]

  # Стоп-слова
  stop_words = set(stopwords.words("russian"))
  for i in range(len(text)):
    text[i] = [word for word in text[i] if not word in stop_words]

  return text

In [ ]:
print(resume_text)
print(vacancy_text)
resume_text = processing(resume_text)
vacancy_text = processing(vacancy_text)
print(resume_text)
print(vacancy_text)

Написал статью, опубликованную в профессиональном журнале. Создание визульных спецэффектов, использование современных технологий работы создания моделей.
Понимания специфики различных материалов и современных технологий нанесения (офсетная и цифровая печать, предпечатная подготовка). Отвечать за разработку визуальных концепций дизайна и их реализацию для различных носителей (полиграфия, текстиль, фарфор и тд).
[['написать', 'статья', 'опубликовывать', 'профессиональный', 'журнал', 'создание', 'визульный', 'спецэффект', 'использование', 'современный', 'технология', 'работа', 'создание', 'модель']]
[['понимание', 'специфика', 'различный', 'материал', 'современный', 'технология', 'нанесение', 'офсетный', 'цифровой', 'печать', 'предпечатный', 'подготовка', 'отвечать', 'разработка', 'визуальный', 'концепция', 'дизайн', 'реализация', 'различный', 'носитель', 'полиграфия', 'текстиль', 'фарфор', 'тд']]


In [ ]:
from gensim.models import Word2Vec

sentences = resume_text + vacancy_text
print(sentences)

# Создание модели Word2Vec
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

model.train(sentences, total_examples=model.corpus_count, epochs=10)

count_similar_words = 0
resume_text_ = resume_text[0]
vacancy_text_ = vacancy_text[0]
for word in resume_text_:
  # Получение вектора слова
  vector = model.wv[word]
  # Поиск похожих слов
  similar_words = model.wv.most_similar(word)
  for similar_word in similar_words:
    if similar_word[0] in vacancy_text_ and similar_word[1] >= 0.15:
      count_similar_words += 1

print(similar_words)
print(count_similar_words)

[['написать', 'статья', 'опубликовывать', 'профессиональный', 'журнал', 'создание', 'визульный', 'спецэффект', 'использование', 'современный', 'технология', 'работа', 'создание', 'модель'], ['понимание', 'специфика', 'различный', 'материал', 'современный', 'технология', 'нанесение', 'офсетный', 'цифровой', 'печать', 'предпечатный', 'подготовка', 'отвечать', 'разработка', 'визуальный', 'концепция', 'дизайн', 'реализация', 'различный', 'носитель', 'полиграфия', 'текстиль', 'фарфор', 'тд']]
[('офсетный', 0.3198152482509613), ('спецэффект', 0.1751776486635208), ('полиграфия', 0.12919537723064423), ('материал', 0.12135910987854004), ('визульный', 0.11133334785699844), ('фарфор', 0.10961601138114929), ('опубликовывать', 0.10607045143842697), ('визуальный', 0.09713415056467056), ('различный', 0.09587042778730392), ('статья', 0.08073121309280396)]
27


In [ ]:
def count_similar_word(resume, vacancy):
  sentences = resume + vacancy
  model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
  model.train(sentences, total_examples=model.corpus_count, epochs=10)
  count = 0
  resume_text = resume[0]
  vacancy_text = vacancy[0]
  for word in resume_text_:
    # Получение вектора слова
    vector = model.wv[word]
    # Поиск похожих слов
    similar_words = model.wv.most_similar(word)
    for similar_word in similar_words:
      if similar_word[0] in vacancy_text and similar_word[1] >= 0.15:
        count += 1

  return count

In [ ]:
print(count_similar_word(resume_text, vacancy_text))

In [ ]:
# Пример словаря
my_dict = {
    'a': 5,
    'b': 2,
    'c': 9,
    'd': 1,
    'e': 7,
    'f': 3,
    'g': 8,
    'h': 6,
    'i': 4,
    'j': 10,
    'k': 0
}

# Сортировка словаря по значениям и получение первых 10 объектов
my_dict = sorted(my_dict.items(), key=lambda item: item[1], reverse=True)[:10]


# Преобразование обратно в словарь (если нужно)
top_10_dict = dict(my_dict)

print(top_10_dict)

{'j': 10, 'c': 9, 'g': 8, 'e': 7, 'h': 6, 'a': 5, 'i': 4, 'f': 3, 'b': 2, 'd': 1}
